In [10]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle

%matplotlib inline

In [11]:
from sidm_orbit_calculation.src.utils.constants import *
from sidm_orbit_calculation.src.plotting.make_plots import *

In [12]:
dpi=175
fontsize=9
pylab.rc('savefig', dpi=dpi)
pylab.rc('text', usetex=True)
pylab.rc('font', size=fontsize)
pylab.rc('xtick.major', pad=5)
pylab.rc('xtick.minor', pad=5)
pylab.rc('ytick.major', pad=5)
pylab.rc('ytick.minor', pad=5)

In [13]:
class Data:
    def __init__(self,fname):
        self.fname = fname
        self.times,self.positions,self.momenta,self.gravity,self.drag,self.density,self.host = self.read_data()
        # self.x0, self.y0, self.px0, self.py0 = self.initial_conditions()

    def read_data(self):
        fi = open(self.fname,'rb')
        data = cPickle.load(fi)
        fi.close()
        return data

    def initial_conditions(self):
        self.x0 = self.positions[0,0]
        self.y0 = self.positions[0,1]
        self.px0 = self.momenta[0,0]
        self.py0 = self.momenta[0,1]
        
    def compute_values(self):
        self.x = self.positions[:,0]
        self.y = self.positions[:,1]
        self.r = np.sqrt(self.x**2+self.y**2)
        self.phi = np.arctan2(self.y,self.x)
        self.px = self.momenta[:,0]
        self.py = self.momenta[:,1]
        self.momentum = np.sqrt(self.px**2+self.py**2)

In [20]:
data_path = '/Users/nora/sidm_orbit_calculation/src/output/'

In [21]:
fname = 'pickle.dat'
data = Data(data_path + fname)
data.initial_conditions()
print 'Initial position = (%.3g, %.3g) m' %(data.x0, data.y0)
print 'Initial velocity = (%.3g, %.3g) m/s' %(data.px0, data.py0)
data.compute_values()

Initial position = (-7.43e+20, 9.97e+21) m
Initial velocity = (-8.04e+05, -7.5e+03) m/s


In [1]:
def select_points(data,nmax=None):
    # select data to plot
    if not nmax: nmax = len(data.times)
    nn = len(data.times)
    tt = data.times[0:nmax:nn/500]
    xx = data.x[0:nmax:nn/500]/data.host.R_200
    yy = data.y[0:nmax:nn/500]/data.host.R_200
    vv = data.px[0:nmax:nn/500]
    return tt, xx, yy, vv

In [16]:
from scipy.special import expi

# ANALYTICAL CALCULATION WITHOUT DRAG
def analytical_velocity_nodrag(x):
    return np.sqrt(2*G*M*(1/x-1/x0))

# ANALYTICAL CALCULATION WITH DRAG
def analytical_velocity_drag(r):
    sigma = 1e4
    c1 = (-G * data.host.M * (data.host.rho * sigma * data.x0 * np.exp(data.host.rho * sigma * data.x0 / 2) * expi(-data.host.rho * sigma * data.x0 / 2) + 2)) / (data.x0 * np.exp(data.host.rho * sigma * data.x0 / 2))
    return np.sqrt(c1 * r * np.exp(data.host.rho * sigma * r / 2) + G * data.host.M * (data.host.rho * sigma * r * np.exp(data.host.rho * sigma * r / 2) * expi(-0.5 * data.host.rho * sigma * r) + 2)) / np.sqrt(r)

In [19]:
def check_velocity(data,drag=False):
    tt, xx, yy, vv = select_points(data,nmax=80000)
    if drag:
        vv_a = analytical_velocity_drag(xx[1:-1]*data.host.R_200)
    else:
        vv_a = analytical_velocity_nodrag(xx*data.host.R_200)

    print 'analytical: (%.5g, %.3g)' %(vv_a.min(),vv_a.max())
    print 'numerical: (%.5g, %.3g)' %(np.abs(vv).min(),np.abs(vv).max())
        
    pylab.figure(figsize=(3,3))
    pylab.plot(tt[1:-1],-vv[1:-1]/1e3,'g-',markersize=10,linewidth=2,label='numerical')
    # pylab.plot(tt2,-vv2/1e3,'r-',markersize=10,linewidth=2,label='no drag')
    pylab.plot(tt[1:-1],vv_a/1e3,'c--',markersize=10,linewidth=2,label='analytical')
    pylab.xlabel(r'$\mathrm{time\ (seconds)}$')
    pylab.ylabel(r'$\mathrm{velocity\ (km/s)}$')
    # pylab.xscale('log')
    pylab.yscale('log')
    pylab.legend(fontsize=8,loc='lower right') # fix formatting
    pylab.show()

In [ ]:
def drag_force():
    fig = plt.figure(figsize=(3,3))
    ax = plt.subplot(111)

    drag_magnitude = np.sqrt(data.drag[:,0]**2+data.drag[:,1]**2)
    ax.plot(xx,drag_magnitude[0:80000:nn/500], 'c', linewidth=2)
    # ax.plot(data.r[:-1]/data.host.R_200,drag_magnitude, 'c', linewidth=2)

    plt.xlabel(r'$\mathrm{Radius\ (r/R_{200})}$')
    plt.ylabel(r'$\mathrm{Effective\ Drag\ Force\ (m/s^2)}$')
    # plt.xscale('log')
    plt.yscale('log')
    
    fig = plt.figure(figsize=(3,3))
    ax = plt.subplot(111)

    ax.plot(-vv*m_to_kpc/seconds_to_years,drag_magnitude[0:80000:nn/500], 'c', linewidth=2)

    plt.xlabel(r'$\mathrm{Particle\ Velocity\ (kpc/yr)}$')
    plt.ylabel(r'$\mathrm{Effective\ Drag\ Force\ (m/s^2)}$')
    plt.xscale('log')
    plt.yscale('log')

    plt.show()

In [ ]:
def radial_profile():
    